You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
    verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
        "3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

2025-08-10 21:12:29,749 - 140146752513728 - __init__.py-__init__:369 - ERROR: Exception while exporting Span batch.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.11/site-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
TimeoutError: timed out

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/urllib3/connectionpool.py", line 793, in urlopen
    response = self._make_request(
               ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/urllib3/connectionpool.py", line 491, in _make_request
    raise new_e
  File 

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Title: The Future of Artificial Intelligence: Trends, Players, and News

Introduction:
- Brief overview of Artificial Intelligence (AI) and its impact on various industries
- Introduction to the latest trends, key players, and noteworthy news in the field of AI

Key Points:
1. Latest Trends in Artificial Intelligence:
- Machine learning advancements
- Natural language processing innovations
- AI ethics and regulation developments
- AI in healthcare, finance, and other industries

2. Key Players in the AI Indu

I now can give a great answer

Final Answer:
# The Future of Artificial Intelligence: Trends, Players, and News

**Introduction:**
Artificial Intelligence (AI) has completely transformed various industries by introducing automation, data analysis, and predictive capabilities. From healthcare to finance, AI has changed the landscape of business operations and technological interactions. This blog post aims to delve into the latest trends, key players, and noteworthy news in the AI sector to offer valuable insights to professionals in the technology, business, and academic realms.

**Latest Trends in Artificial Intelligence:**
The forefront of AI innovation has been dominated by advancements in machine learning, enabling systems to enhance their performance by learning from data. Additionally, breakthroughs in natural language processing have significantly improved communication between humans and machines. The establishment of ethical guidelines and regulations for AI is crucial to ensu

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Trends, Players, and News

**Introduction:**
Artificial Intelligence (AI) has completely transformed various industries by introducing automation, data analysis, and predictive capabilities. From healthcare to finance, AI has changed the landscape of business operations and technological interactions. This blog post aims to delve into the latest trends, key players, and noteworthy news in the AI sector to offer valuable insights to professionals in the technology, business, and academic realms.

**Latest Trends in Artificial Intelligence:**
The forefront of AI innovation has been dominated by advancements in machine learning, enabling systems to enhance their performance by learning from data. Additionally, breakthroughs in natural language processing have significantly improved communication between humans and machines. The establishment of ethical guidelines and regulations for AI is crucial to ensure responsible and ethical deployment of AI solutions in industries like healthcare and finance.

**Key Players in the AI Industry:**
Leading the charge in AI research and development are major players such as Google DeepMind, IBM Watson, Microsoft Azure AI, Amazon Web Services (AWS) AI, OpenAI, Nvidia, and Intel AI. These companies have made substantial contributions to the field, pushing the boundaries of artificial intelligence technologies and driving innovation.

**Noteworthy News in Artificial Intelligence:**
Recent advancements in AI research have opened up new possibilities for addressing real-world challenges. Collaborations and partnerships within the AI industry have resulted in cutting-edge solutions for autonomous vehicles, robotics, and other domains. Remaining updated on the latest developments in AI is crucial for professionals seeking to leverage AI's potential in their respective fields.

**Call to Action:**
As AI continues to progress rapidly, professionals must actively stay informed and engaged with the latest trends and news in the field. By staying abreast of AI advancements, individuals can make informed decisions and spur innovation within their organizations. Let's embrace the promising future of artificial intelligence and explore the myriad opportunities it presents.

In conclusion, the future of artificial intelligence looks promising, with continuous advancements, influential players shaping the industry, and groundbreaking news driving innovation. By staying informed and engaged with the evolving trends in AI, professionals can harness the power of artificial intelligence to propel growth and success in their fields. Stay tuned for more updates on the dynamic world of artificial intelligence.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "Máquina da Gödel"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Máquina da Gödel.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan: 
Title: Unveiling the Mysteries of Máquina da Gödel

Introduction:
- Brief overview of Máquina da Gödel and its significance in the field of computer science and mathematics.
- Highlight the complexity and implications of Gödel's incompleteness theorems in relation to the machine.

Key Points:
1. The History of Máquina da Gödel
- Discuss the origins of the concept and its development over time.
- Explain how the machine operates and its theoretical capabilities.

2. Latest Trends and Noteworthy News
- 

I now can give a great answer
Final Answer: 

# Unveiling the Mysteries of Máquina da Gödel

## Introduction

The Máquina da Gödel, also known as the Gödel Machine, is a captivating concept that merges the realms of computer science and mathematics, particularly within the context of Gödel's incompleteness theorems. These theorems, formulated by the esteemed mathematician Kurt Gödel, hold profound implications for the boundaries of formal systems and computational logic. The Máquina da Gödel serves as a theoretical construct that embodies the essence of Gödel's theorems, pushing the limits of what is computationally feasible.

## The History of Máquina da Gödel

The roots of the Máquina da Gödel can be traced back to Gödel's groundbreaking work on the incompleteness theorems in the early 20th century. This machine, essentially, is a hypothetical device capable of executing any mathematical computation based on a given set of axioms. It functions by iteratively enhancing its own code, p

In [14]:
Markdown(result)

# Unveiling the Mysteries of Máquina da Gödel

## Introduction

The Máquina da Gödel, also known as the Gödel Machine, is a captivating concept that merges the realms of computer science and mathematics, particularly within the context of Gödel's incompleteness theorems. These theorems, formulated by the esteemed mathematician Kurt Gödel, hold profound implications for the boundaries of formal systems and computational logic. The Máquina da Gödel serves as a theoretical construct that embodies the essence of Gödel's theorems, pushing the limits of what is computationally feasible.

## The History of Máquina da Gödel

The roots of the Máquina da Gödel can be traced back to Gödel's groundbreaking work on the incompleteness theorems in the early 20th century. This machine, essentially, is a hypothetical device capable of executing any mathematical computation based on a given set of axioms. It functions by iteratively enhancing its own code, potentially leading to infinite self-improvement in solving mathematical problems. The theoretical capacities of the Máquina da Gödel are extensive, demonstrating the power and intricacy of computational logic inspired by Gödel's theorems.

## Latest Trends and Noteworthy News

In recent times, progressions in computational logic and artificial intelligence have sparked renewed interest in the principles underpinning the Máquina da Gödel. Researchers and academics are exploring novel pathways in theoretical computer science, delving deep into Gödel's theorems to expand the boundaries of what is attainable in computation. Breakthroughs associated with the Máquina da Gödel are paving the way for inventive solutions in logic-based issue resolution and artificial intelligence applications.

## Key Players in the Field

Prominent figures in the realm of computational logic and Gödel's theorems have played pivotal roles in shaping our comprehension of the Máquina da Gödel. Influential researchers and scholars, such as Jürgen Schmidhuber, have made significant contributions to the advancement of this theoretical machine, elucidating the intricate connections between mathematics, philosophy, and technology. Their endeavors continue to inspire new cohorts of students and professionals in the domains of computer science, mathematics, and artificial intelligence.

In conclusion, the Máquina da Gödel stands as a testament to the enduring legacy of Kurt Gödel and his incompleteness theorems. By delving into the intricacies of computational logic and theoretical computer science, we embark on a voyage of exploration and innovation that challenges the very essence of what is computationally achievable. As we investigate the confluence of philosophy, mathematics, and technology through the prism of the Máquina da Gödel, we unearth fresh horizons of knowledge and insight in this ever-evolving field.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).